In [ ]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from datetime import date

In [ ]:
# GRUPO 1
def listar_publicacoes(data):
    
    options = Options()
    options.add_argument('--headless')
    
    driver = webdriver.Chrome(options=options)

    url = 'https://www.in.gov.br/leiturajornal'

    data = data
    orgao = 'Ministério-da-Saúde'
    orgao_sub = 'Agência-Nacional-de-Vigilância-Sanitária'
    ato = 'Resolução'

    url_base = f'https://www.in.gov.br/leiturajornal?secao=dou1&data={data}&org={orgao}&org_sub={orgao_sub}&ato={ato}'

    driver.get(url_base)
               
    html_content = driver.page_source

    driver.quit()

    soup = BeautifulSoup(html_content, 'html.parser')

    padrao = re.compile(r'/web/dou/-/resolucao.*')
    links = soup.find_all('a', href=padrao)

    links_filtrados = [] 
    for link in links:
        li = link.get('href')
        links_filtrados.append(f'https://www.in.gov.br/{li}')
       
        
    return(links_filtrados)

In [ ]:
def extrair_informacoes(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # GRUPO 2
    publicado_em_elemento = soup.find('span', {'class': 'publicado-dou-data'})
    publicado_em = publicado_em_elemento.text.strip() if publicado_em_elemento is not None else None
    
    edicao_elemento = soup.find('span', {'class': 'edicao-dou-data'})
    edicao = edicao_elemento.text.strip() if edicao_elemento is not None else None
    
    secao_elemento = soup.find('span', {'class': 'secao-dou'})
    secao = secao_elemento.text.strip() if secao_elemento is not None else None
    
    pagina_elemento = soup.find('span', {'class': 'secao-dou-data'})
    pagina = pagina_elemento.text.strip() if pagina_elemento is not None else None
    
    orgao_elemento = soup.find('span', {'class': 'orgao-dou-data'})
    orgao = orgao_elemento.text.strip() if orgao_elemento is not None else None
    
    assina_elemento = soup.find('p', {'class': 'assina'})
    assina = assina_elemento.text.strip() if assina_elemento is not None else None
    
    identifica_elemento = soup.find('p', {'class': 'identifica'})
    identifica = identifica_elemento.text.strip() if identifica_elemento is not None else None
    
    preambulo_elemento = soup.find('p', {'class': 'dou-paragraph'})
    preambulo = preambulo_elemento.text.strip() if preambulo_elemento is not None else None
    
    versao_oficial_elemento = soup.find('a', {'id': 'versao-certificada'})
    versao = versao_oficial_elemento.get('href') if versao_oficial_elemento is not None else None
    
    artigos = re.compile(r'Art\.\s?\d+º')
    paragrafos_artigos = []

    for paragraph in soup.find_all('p'):
        if artigos.search(paragraph.text):
            paragrafos_artigos.append(paragraph.text.strip())

    #GRUPO 3
    #Extrair número do Expediente
    expediente_elements = soup.find_all('p', text=re.compile(r'Expediente'))
    numeros_expediente = []
    for expediente_element in expediente_elements:
        expediente_text = expediente_element.text.strip()
        numero_expediente = re.search(r'(\d+/\d+-\d)', expediente_text)
        if numero_expediente:
            numeros_expediente.append(numero_expediente.group(1))

    #Extrair Texto completo
    completo = []
    for i in soup.find_all('p', {'class':'dou-paragraph'}):
        completo.append(i.text.strip())
       
    #Ajuste fino
       
    data = {
    'DT_PUBLICACAO': [publicado_em],
    'EDICAO': [edicao],
    'SECAO': [secao],
    'PAGINA': [pagina],
    'ORGAO':[orgao],
    'ASSINATURA':[assina],
    'PREAMBULO':[preambulo],
    'ARTIGOS':[paragrafos_artigos],
    'IDENTIFICACAO':[identifica], 
    'VERSAO_OFICIAL': [versao],
    'EXPEDIENTE': [numeros_expediente],
    'TEXTO_COMPLETO':[completo]
    }

    df = pd.DataFrame(data)
    return(df)